<a href="https://colab.research.google.com/github/lephuocdat2000/DeepLearning-and-Application/blob/main/EngtoVie_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing Data

###1, Import libraries

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd



###2, Reading the data

In [3]:
!wget http://www.manythings.org/anki/vie-eng.zip -O viet-eng.zip
!unzip viet-eng.zip

--2021-06-20 02:28:07--  http://www.manythings.org/anki/vie-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297648 (291K) [application/zip]
Saving to: ‘viet-eng.zip’

viet-eng.zip        100%[===================>] 290.67K  1.80MB/s    in 0.2s    

2021-06-20 02:28:08 (1.80 MB/s) - ‘viet-eng.zip’ saved [297648/297648]

Archive:  viet-eng.zip
  inflating: _about.txt              
  inflating: vie.txt                 


In [4]:
lines = pd.read_table( 'vie.txt' , names=[ 'eng' , 'vie' ] )
lines.reset_index( level=0 , inplace=True)
lines.rename( columns={ 'index' : 'eng' , 'eng' : 'vie' , 'vie' : 'c' } , inplace=True )
lines = lines.drop('c',1)

In [5]:
lines.head()

,eng,vie
0,Run!,Chạy!
1,Help!,Giúp tôi với!
2,Go on.,Tiếp tục đi.
3,Hello!,Chào bạn.
4,Hurry!,Nhanh lên nào!


### 3) Preparing input data for the Encoder ( `encoder_input_data` )

In [12]:
eng_lines = list()
for line in lines.eng:
    eng_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()  
tokenizer.fit_on_texts( eng_lines ) 
tokenized_eng_lines = tokenizer.texts_to_sequences( eng_lines ) 

length_list = list()
for token_seq in tokenized_eng_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
padded_eng_lines = preprocessing.sequence.pad_sequences( tokenized_eng_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_eng_lines )

### 4) Preparing input data for the Decoder ( `decoder_input_data` )

In [17]:
!pip install underthesea

     |████████████████████████████████| 7.5MB 8.6MB/s 
     |████████████████████████████████| 245kB 43.8MB/s 
     |████████████████████████████████| 1.3MB 33.8MB/s 
     |████████████████████████████████| 747kB 43.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 901kB 31.2MB/s 
     |████████████████████████████████| 2.9MB 26.3MB/s 
     |████████████████████████████████| 1.1MB 21.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=3502f4239294b1b2dd604af444e06a48c3f2e3c2ac53a2c6a1fe14f623dfd6fc
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.1 which is incompat

In [105]:
from underthesea import word_tokenize
tokenized_vie_lines = []
dict_words = np.array([])
dict_numbers = np.array([]).astype('int64')
vie_lines = []
tokenized_vie_lines = []

for line in lines.vie:
    line = word_tokenize(line,format='text')  
    full_line = '<START> ' + line + ' <END>'
    words = full_line.split(" ")
    for word in words:
       where_word_arr = np.where(dict_words==word)[0]
       if len(where_word_arr)==0: 
           dict_words=np.append(dict_words,word)
           dict_numbers=np.append(dict_numbers,1)
       else: 
         dict_numbers[where_word_arr[0]]+=1
    vie_lines.append(full_line)

agr_sorted_dict_numbers = np.argsort(dict_numbers)[::-1]

# tokenizer = preprocessing.text.Tokenizer()
# tokenizer.fit_on_texts(vie_lines) 
# tokenized_vie_lines = tokenizer.texts_to_sequences(vie_lines) 

# length_list = list()
# for token_seq in tokenized_vie_lines:
#     length_list.append( len( token_seq ))

# max_output_length = np.array( length_list ).max()
# print( 'Vietnamese max length is {}'.format( max_output_length ))

# padded_vie_lines = preprocessing.sequence.pad_sequences( tokenized_vie_lines , maxlen=max_output_length, padding='post' )
# decoder_input_data = np.array( padded_vie_lines )
# print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


In [133]:
for i,vie_line in enumerate(vie_lines):
    words = vie_line.split(" ")
    words_arr = np.zeros(len(words))
    for i,word in enumerate(words):
       where_word = np.where(dict_words==word)[0][0] 
       temp =np.where(agr_sorted_dict_numbers==where_word)[0][0] +1
       words_arr[i] = format(temp,'.0f') 
    tokenized_vie_lines.append(words_arr)

In [134]:
print(vie_lines[1][0])
tokenized_vie_lines[1][1]

<START> Giúp tôi với ! <END>


array([1.000e+00, 2.868e+03, 9.000e+00, 3.600e+01, 1.020e+02, 2.000e+00])

In [37]:
vie_word_dict = tokenizer.word_index
num_vie_tokens = len(vie_word_dict )+1
print( 'Number of Vietnamese tokens = {}'.format( num_vie_tokens))

Number of Marathi tokens = 2361


In [47]:
print(type(tokenized_vie_lines))

<class 'list'>


In [84]:
tokenized_eng_lines

[[630],
 [96],
 [41, 32],
 [1313],
 [512],
 [1, 163],
 [1, 163],
 [125, 15],
 [125, 15],
 [98, 15],
 [96, 13],
 [1, 631],
 [1085],
 [25, 38],
 [25, 38],
 [5, 630],
 [1632, 56],
 [9, 1314],
 [9, 1314],
 [512, 56],
 [1, 698],
 [80, 163],
 [80, 163],
 [22, 1315],
 [22, 169],
 [99, 236],
 [73, 50],
 [73, 50],
 [73, 50],
 [41, 2, 329],
 [1, 451, 330],
 [1, 835],
 [1, 835],
 [1, 42, 41],
 [7, 15, 282],
 [57, 290],
 [57, 1316],
 [57, 1316],
 [166, 129],
 [166, 129],
 [220, 50],
 [23, 836],
 [23, 836],
 [452, 45],
 [151, 56],
 [151, 56],
 [21, 5, 580],
 [221, 6, 316],
 [512, 77],
 [1, 39, 254],
 [1, 581, 97],
 [1, 581, 97],
 [1, 451, 5],
 [1, 161, 5],
 [1, 161, 5],
 [1, 42, 163],
 [1, 42, 163],
 [22, 1086],
 [22, 97, 1317],
 [22, 97, 1317],
 [57, 2294],
 [147, 13, 163],
 [54, 837],
 [47, 7, 56],
 [47, 7, 56],
 [47, 7, 56],
 [4, 1633],
 [4, 1633],
 [453, 50],
 [29, 2295],
 [17, 838],
 [17, 838],
 [17, 196],
 [17, 196],
 [17, 196],
 [17, 196],
 [41, 2, 211],
 [1, 268, 5],
 [1, 34, 173],
 [1, 34,